In [ ]:
%matplotlib inline
import torch as t
import os
import numpy as np
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset
import pandas as pd
import re
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable

In [ ]:
import moxing as mox
mox.file.shift('os', 'mox')

In [ ]:
transform = T.Compose([
    T.Resize(224),#大小重设
    T.CenterCrop(224),#可以改成不固定中心点的函数
    T.ToTensor(),#正则化
    #T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

class DataClassify(Dataset):
    def __init__(self, root, transforms=None, mode=None):
        #存放图像地址
        self.imgs = [x.path for x in mox.file.scan_dir(root) if
            x.name.endswith(".jpg")]
        self.labels = [y.path for y in mox.file.scan_dir(root) if
            y.name.endswith(".txt")]
        self.transforms = transforms
        
    def __getitem__(self, index):
        #读取图像数据并返回
        img_path = self.imgs[index]
        #label = open(self.labels[index]).read()
        #label = re.sub('\D','',open(self.labels[index]).read()[-4:])
        label = int(re.sub('\D','',mox.file.read(self.labels[index])[-4:]))
        data = Image.open(img_path)
        if self.transforms:
            data = self.transforms(data)
        return data, label
    
    def __len__(self):
        return len(self.imgs)
    
dataset = DataClassify('s3://classifier-gar/try/', transforms=transform)
#for data in dataset:
#    X_train, X_label = data
#    print(X_train.type(), X_label)
#img, label = dataset[0]##dataset[0]:读取数据集首个元素
#for img, label in dataset:
#   print(img.size(), label)

In [ ]:
#-------------------------------------
print("data loading...")
#-------------------------------------
data_loader_train = t.utils.data.DataLoader(dataset=dataset, batch_size=64, shuffle=False)
#-------------------------------------
print("data load complete")
#-------------------------------------

In [ ]:
#dataiter = iter(data_loader_train)
#data, label = next(dataiter)
#data.type()
#print(data.size())
#t_label = []
#for l in label:
#    t_label.append(l)

def ListToTensor(lister):
    nparr = np.asarray(lister)
    tens = t.from_numpy(nparr)
    return tens

In [ ]:
class ResBlock(nn.Module):
    #残差块
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, 3, stride, 1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, 3, 1, 1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut
        
    def forward(self, x):
        out = self.left(x)
        
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_classes=40):
        super(ResNet, self).__init__()
        #--------------------------------
        print("残差网络搭建中...")
        #--------------------------------
        self.pre = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 1)
        )
        #--------------------------------
        print("pre层搭建成功...")
        #--------------------------------
        self.layer1 = self._make_layer(64, 128, 3)
        #--------------------------------
        print("第一层搭建成功...")
        #--------------------------------
        self.layer2 = self._make_layer(128, 256, 4, stride=2)
        #--------------------------------
        print("第二层搭建成功...")
        #--------------------------------
        self.layer3 = self._make_layer(256, 512, 6, stride=2)
        #--------------------------------
        print("第三层搭建成功...")
        #--------------------------------
        self.layer4 = self._make_layer(512, 512, 3, stride=2)
        #--------------------------------
        print("第四层搭建成功...")
        #--------------------------------
        
        self.fc = nn.Linear(512, num_classes)
        #--------------------------------
        #print("全连接层搭建成功...")
        #--------------------------------
    
    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        #构建包含多个残差块的layer
        shortcut = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, 1, stride, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)
        #-------------------------------------
        print("pre层通过")
        #-------------------------------------
        x = self.layer1(x)
        #-------------------------------------
        print("第一层通过")
        #-------------------------------------
        x = self.layer2(x)
        #-------------------------------------
        print("第二层通过")
        #-------------------------------------
        x = self.layer3(x)
        #-------------------------------------
        print("第三层通过")
        #-------------------------------------
        x = self.layer4(x)
        #-------------------------------------
        print("第四层通过")
        #-------------------------------------
        
        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
net = ResNet()
cost = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(net.parameters(), lr=0.001)

In [ ]:
print(net)

In [ ]:
epochs = 2
#img, label = dataset[0]##dataset[0]:读取数据集首个元素
#for img, label in dataset:
#   print(img.size(), label)
for epoch in range(epochs):
    print("epoch " + str(epoch) + " start training...")
    run_loss = 0.0
    run_currect = 0
    
    for data in iter(data_loader_train):
        X_train, X_label = data
        label = []
        for la in X_label:
            label.append(la)
        label = ListToTensor(label)
        
        X_train = Variable(X_train)
        X_label = Variable(label)
        outputs = net(X_train)
        _, pred = t.max(outputs.data, 1)
        optimizer.zero_grad()
        loss = cost(outputs, X_label)
        
        loss.backward()
        optimizer.step()
        run_loss += t.sum(pred == X_label.data)
        print('epoch %d, loss %.2f, currect %.2f ------' %(epoch+1, run_loss, run_currect))
